In [1]:
import pandas as pd
from shapely.wkt import loads
from folium.plugins import HeatMap
from shapely.geometry import mapping
from datetime import datetime, timedelta


In [2]:

def get_portugal_holidays(start_year, end_year):
    portugal_holidays = []
    for year in range(start_year, end_year + 1):
        holidays = [
            f"{year}-01-01",
            f"{year}-04-14",  # Good Friday
            f"{year}-04-16",  # Easter Sunday
            f"{year}-04-25",
            f"{year}-05-01",
            f"{year}-06-10",
            f"{year}-06-15",  # Corpus Christi
            f"{year}-08-15",
            f"{year}-10-05",
            f"{year}-11-01",
            f"{year}-12-01",
            f"{year}-12-25"
        ]
        portugal_holidays.extend([datetime.strptime(date, "%Y-%m-%d") for date in holidays])

    return portugal_holidays

start_year = 2021  # Change this to the starting year
end_year = 2023    # Change this to the ending year

portugal_holidays = get_portugal_holidays(start_year, end_year)

In [3]:
def get_time_of_day(hour):
    if 7 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 19:
        return 'Afternoon'
    else:
        return 'Night'

def is_weekend(day):
    return day.weekday() >= 5  

def is_holiday(day):
    return day in portugal_holidays


In [6]:
file_path = 'January2023.csv' 
file_path_e = 'wktComplete.csv'

columns_to_keep_vodafone = ['Grid_ID','Datetime','C1', 'C3', 'C5', 'C6', 'E7', 'E8', 'E9']
columns_to_keep_wkt = ["grelha_id", 'nome','latitude','longitude']

df_dados_vodafone = pd.read_csv(file_path, encoding='latin1',usecols=columns_to_keep_vodafone)
df_wkt = pd.read_csv(file_path_e,encoding='latin1',usecols=columns_to_keep_wkt)

df_wkt = df_wkt.rename(columns={'grelha_id': 'Grid_ID'}) 

merged_df = pd.merge(df_dados_vodafone, df_wkt, on='Grid_ID')

merged_df = merged_df.drop(0)

In [7]:
merged_df

merged_df['datetime_column'] = pd.to_datetime(merged_df["Datetime"])

# Separate into date and hour columns
merged_df['date'] = merged_df['datetime_column'].dt.date
merged_df['time'] = merged_df['datetime_column'].dt.time




merged_df['time_of_day'] = merged_df['datetime_column'].dt.hour.apply(get_time_of_day)
merged_df['is_weekend'] = merged_df['datetime_column'].dt.date.apply(is_weekend)
merged_df['is_holiday'] = merged_df['datetime_column'].dt.date.apply(is_holiday)


df = merged_df.drop(columns=['datetime_column'])


In [8]:
#patterns = ['A1', 'Eixo Norte-Sul', 'CRIL', 'Segunda Circular', '2ª Circular']
patterns = ['IC19','Segunda Circular', '2ª Circular','Eixo Norte-Sul']
numbers_list = [2548, 2549, 2608,2616,2617, 2670, 2731, 2792,2816,2214,2344,2081, 2852, 2911, 2972,3031,2973, 3032, 3091, 3092,3417, 3546, 3581, 3613, 3612, 3637, 3659, 3660, 3661, 3662, 3680, 3681, 3697, 2477, 2543, 2542, 2411, 1841, 1840, 1779, 1778, 1302, 1303, 1241, 1242, 1182, 1124, 1066, 1009, 953, 896, 839
]

combined_pattern = '|'.join(patterns)

merged_df_truncated1 = df[df['Grid_ID'].isin(numbers_list)]


merged_df_truncated2 = df[df.nome.str.contains(combined_pattern, regex=True)]

concatenated_df = pd.concat([merged_df_truncated1, merged_df_truncated2], ignore_index=True)

df_no_duplicates = concatenated_df.drop_duplicates()

df_no_duplicates.to_csv("January2023Clean.csv")

In [40]:
len(df_no_duplicates)

362880

In [ ]:
len()

In [ ]:
del df_dados_vodafone, df_wkt, merged_df, patterns, combined_pattern